## โค้ดสร้าง CAPPI ด้วยการใช้ PPI ในแต่ละมุมยก

In [1]:
import os
import numpy as np
import xarray as xr
import rioxarray
import pyart
from datetime import datetime
from dateutil import tz
import copy  
import warnings
warnings.filterwarnings("ignore")

def utc2local_hr(time_rad):
    from_zone = tz.tzutc()
    to_zone = tz.tzlocal()
    utc = datetime.strptime(time_rad, "%Y%m%d%H")
    utc = utc.replace(tzinfo=from_zone)
    lst = utc.astimezone(to_zone)
    return lst

def snr_atten_cor(radar):
    snr = pyart.retrieve.calculate_snr_from_reflectivity(radar, refl_field='reflectivity', toa=15000.0)
    radar.add_field('signal_to_noise_ratio', snr, replace_existing=True)
    gtfilter = pyart.filters.moment_and_texture_based_gate_filter(radar, phi_field='differential_phase')
    gtfilter.exclude_below('signal_to_noise_ratio', 5)
    gtfilter.exclude_above('signal_to_noise_ratio', 70)
    radar.add_field_like('reflectivity', 'reflectivity_copy', radar.fields['reflectivity']['data'].copy())
    nf = radar.fields['reflectivity_copy']
    nf['data'] = np.ma.masked_where(gtfilter.gate_excluded, nf['data'])
    radar.add_field('filtered_reflectivity', nf, replace_existing=True)

    ncp_values = np.ones((radar.nrays, radar.ngates))
    ncp = pyart.config.get_metadata('normalized_coherent_power')
    ncp['data'] = ncp_values
    radar.add_field('normalized_coherent_power', ncp)

    phidp, kdp = pyart.correct.phase_proc_lp(radar, 0.0, LP_solver='pyglpk', debug=True)
    radar.add_field('proc_dp_phase_shift', phidp)
    radar.add_field('recalculated_diff_phase', kdp)

    spec_at, cor_z = pyart.correct.calculate_attenuation(
        radar,
        0,
        fzl=4500.0,
        refl_field="filtered_reflectivity",
        ncp_field="normalized_coherent_power",
        rhv_field="cross_correlation_ratio",
        phidp_field="proc_dp_phase_shift",
    )
    radar.add_field("specific_attenuation", spec_at)
    radar.add_field("corrected_filtered_reflectivity", cor_z)

    return radar

## 4. ฟังก์ชั่นสร้างกริด CAPPI จาก PPI แต่ละมุมยกที่ต้องการ
def gridding_PPI_convertGrid2Xaaray(radar, sweep_num):
    radar_p=radar.extract_sweeps([sweep_num]) # เลือกมุมยก 0 คือ มุมแรก
    lat_0 = radar_p.latitude['data'][0]
    lon_0 = radar_p.longitude['data'][0]

    shape = (1, 241, 241)
    grid = pyart.map.grid_from_radars(
                radar_p,
                grid_shape=shape, #Number of points in the grid (z, y, x)
                grid_limits=((sweep_num*1000, sweep_num*1000), (-240000, 240000), (-240000, 240000)),    # ตั้งค่า(2000, 2000) หมายความว่าต้องการให้สร้างกริด cappi ที่ระดับ 2 km จาก ppi ดังกล่าว
                grid_origin = (lat_0,lon_0),
                fields=['corrected_filtered_reflectivity'],
                roi_func= 'constant', constant_roi=4000.0 # เปลี่ยนฟังก์ชั่นเป็น roi_func= 'constant' เพื่อต้องการให้ดึงขัอมูลจุดของมุม ppi ที่อยู่ในระดับสูงมาคิดน้ำหนักด้วย สังเกตตรงกลางสถานีจะมีค่าฝนแล้ว
    )
    
    # แปลง grid ใน pyart ไปสู่ข้อมูล xaaray
    ds = grid.to_xarray()
    grid =ds['corrected_filtered_reflectivity'][0,0,:,:] #เวลา 0,0 คือ มีเวลาเดียว และมีระดับความสูงเดียว
    
    return grid

def save_geotif(cappi_max_hr, path_outfile):
    cappi_max_hr = cappi_max_hr.to_dataset().squeeze().set_index(x="lon", y="lat")
    cappi_max_hr.rio.set_spatial_dims(x_dim='x', y_dim='y', inplace=True)
    cappi_max_hr.rio.set_crs("EPSG:4326")
    cappi_max_hr.rio.write_crs("EPSG:4326", inplace=True)
    cappi_max_hr.rio.write_transform()
    cappi_max_hr.rio.write_coordinate_system()
    cappi_max_hr.rio.to_raster(path_outfile)
    cappi_max_hr.close()

def main():
    dir_path = '../../0data/0radar/0test_phs_1hour/'
    #dir_path = '../../0data/0radar/0test_phs_5hour/'
    path_results = './0results/0cappi_ppi_hourly_test1/ppi/'

    fn_all = [file for file in os.listdir(dir_path) if file.endswith('.uf')]
    fn_hr = list(np.unique([file[3:-7] for file in fn_all]))

    # Read and preprocess radar data for SNR and attenuation correction
    radar_dict = {}
    print('#1.> ปรับแก้ SNR and attenuation correction...')
    for sub_file_hr in fn_hr:
        files_15min = [file for file in fn_all if sub_file_hr in file]
        
        # Initialize radar_list to store radar data for all files in the current hour
        radar_list = []
        
        print('#---> processing '+sub_file_hr+' ...>>>')
        for file_15min in files_15min:
            radar = pyart.io.read_uf(os.path.join(dir_path, file_15min))
            radar.metadata['file_name'] = file_15min  # Store file name as metadata เก็บไว้เพื่อ print
            radar = snr_atten_cor(radar)
            radar_list.append(radar)

        # Add the list of corrected radar data to the dictionary with the hour as the key
        radar_dict[sub_file_hr] = radar_list

    # Specify the heights you want to generate CAPPI for
    sweeps = [0, 1, 2, 3]
    print('#2.> สร้างแผนที่ PPIs แต่ละมุมยก....')
    for sweep in sweeps:
        for sub_file_hr, radar_list in radar_dict.items():
            local_time = utc2local_hr(sub_file_hr).strftime('%Y%m%d%H')
            print('---------------------------------------\n')
            print('Find max for .......>>>', sub_file_hr)

            con_xr = []

            for file_15min, radar in zip(files_15min, radar_list):
                #print(f'Doing ...>>> {radar.metadata}') #file_15min อันนี้ไม่เวิร์คนะ ชื่อไม่ตรงกับที่ประมวลผล ต้องลองสัก 2 ชั่วโมงดู
                print('.......>>>', radar.metadata['file_name'])

                # Reuse the corrected radar data for different heights
                radar_copy = copy.deepcopy(radar)  # Use deepcopy instead of copy

                print(f'>> Do gridding PPI for sweep {sweep} ...')
                grid = gridding_PPI_convertGrid2Xaaray(radar_copy, sweep)
                con_xr.append(grid)

            print(f'+ Do find max of PPI at sweep {sweep} ...')
            cappi_con_xr = xr.concat(con_xr, dim='time')
            cappi_max_hr = cappi_con_xr.max(dim='time')

            print(f'+ Do save results for PPI at sweep {sweep} ...')

            # Specify the height for folder creation
            sweep_folder = sweep+1 #zero-based ต้อง +1
            folder_name = f'ppi{sweep_folder}'

            # Create the folder if it doesn't exist
            folder_path = os.path.join(path_results, folder_name)
            os.makedirs(folder_path, exist_ok=True)

            outp_name = local_time + '.tif'
            save_geotif(cappi_max_hr, os.path.join(folder_path, outp_name))
            print(f'---------------Saved results for PPI at sweep {sweep} ---------------\n')

if __name__ == "__main__":
    main()



## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead


#1.> ปรับแก้ SNR and attenuation correction...
#---> processing 2018072102 ...>>>
Unfolding
Exec time:  1.5599398612976074
Doing  0
Doing  1
Doing  2
Doing  3
Unfolding
Exec time:  1.502135992050171
Doing  0
Doing  1
Doing  2
Doing  3
Unfolding
Exec time:  1.5135259628295898
Doing  0
Doing  1
Doing  2
Doing  3
Unfolding
Exec time:  1.5527355670928955
Doing  0
Doing  1
Doing  2
Doing  3
#2.> สร้างแผนที่ PPIs แต่ละมุมยก....
---------------------------------------

Find max for .......>>> 2018072102
Doing ...>>> PHI201807210200f2.uf
>> Do gridding PPI for sweep 0 ...
Doing ...>>> PHI201807210215f3.uf
>> Do gridding PPI for sweep 0 ...
Doing ...>>> PHI201807210230f4.uf
>> Do gridding PPI for sweep 0 ...
Doing ...>>> PHI201807210245f5.uf
>> Do gridding PPI for sweep 0 ...
+ Do find max of PPI at sweep 0 ...
+ Do save results for PPI at sweep 0 ...
---------------Saved results for PPI at sweep 0 ---------------

---------------------------------------

Find max for .......>>> 2018072102
Doin